In [1]:
# A GRAPHICS CARD ID NEEDED TO RUN THIS SCRIPT #

In [ ]:
import os

In [ ]:
from tqdm.autonotebook import tqdm
from pprint import pprint
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

from sklearn.metrics import classification_report

from simple_classification_model import SimpleCNN

device = torch.device('cuda:0')

In [ ]:
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

n_channels = 3
n_classes = 9

In [ ]:
import torch
from torchvision import datasets, transforms
import helper

In [ ]:

data_transform = transforms.Compose([transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

PATH_train = 'output/train'
PATH_test = 'output/test'

train_dataset = datasets.ImageFolder(PATH_train, transform=data_transform)
test_dataset = datasets.ImageFolder(PATH_test, transform=data_transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
images, labels = next(iter(trainloader))
helper.imshow(images[0], normalize=False)

In [ ]:
images, labels = next(iter(testloader))
helper.imshow(images[0], normalize=False)

In [ ]:
data_iter = iter(trainloader)
labels = next(data_iter)

In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)


## creating the model ##

In [ ]:
model = SimpleCNN(in_channels=n_channels, num_classes=n_classes).to(device)

criterion = nn.CrossEntropyLoss()
    
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

## training the model ##

In [ ]:
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    
    # set the model into train mode
    model.train()

    for inputs, targets in tqdm(trainloader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)
        
        #if task == 'multi-label, binary-class':
            #targets = targets.to(torch.float32)
            #loss = criterion(outputs, targets)
        #else:
            #targets = targets.squeeze().long()
            #loss = criterion(outputs, targets)
        #print(targets)
        #targets = targets.unsqueeze(1).long()
        #targets = targets.to(torch.float32)
        #targets = targets.unsqueeze(1).long()

        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

## evaluating the model ##

In [ ]:
def test(split):
    model.eval()
    y_true = torch.tensor([]).to(device)
    y_score = torch.tensor([]).to(device)

    with torch.no_grad():
        for inputs, targets in testloader:
            outputs = model(inputs.to(device))

            targets = targets.to(torch.float32).to(device)
            outputs = outputs.softmax(dim=-1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.cpu().numpy()
        y_score = y_score.detach().cpu().numpy()
        
        print('-' * 10)
        print(classification_report(y_true.squeeze(), y_score.argmax(axis=1)))
  

print('==> Evaluating ...')
test('train')
test('test')